In [ ]:
import matplotlib.pyplot as plt
from pylsl import StreamInlet, resolve_byprop
import numpy as np
import time
from IPython.display import display, clear_output

# Look for Aurora fNIRS LSL stream
print("Looking for Aurora fNIRS LSL stream...")
fnirs_streams = resolve_byprop('name', 'Aurora', timeout=20)
if not fnirs_streams:
    print("No 'Aurora' LSL stream found. Make sure Aurora is streaming.")
    exit()

# Look for the LSL marker stream (sent by Python Script #1)
print("Looking for LSL marker stream...")
# marker_streams = resolve_byprop('type', 'Markers', timeout=20)
marker_streams = resolve_byprop('name', 'WebSocketLSL', timeout=20)
if not marker_streams:
    print("No marker stream found. Make sure Python Script #1 is running.")
    exit()

# Connect to LSL streams
fnirs_inlet = StreamInlet(fnirs_streams[0])
marker_inlet = StreamInlet(marker_streams[0])

print(f"Connected to fNIRS stream: {fnirs_streams[0].name()}")
print(f"Connected to Marker stream: {marker_streams[0].name()}")

# Setup plot
plt.ion()
fig, ax = plt.subplots()
hbo_data, hbr_data, timestamps = [], [], []
recording = False  # Flag to track if we are recording fNIRS data
no_data_count = 0  
max_no_data_attempts = 5  

# Wait for first marker
print("Waiting for marker '12' to start recording...")
while not recording:
    marker_sample, marker_timestamp = marker_inlet.pull_sample(timeout=2.0)
    if marker_sample and marker_sample[0] == 12:
        print(f"Marker '12' received at {marker_timestamp}, starting fNIRS data collection!")
        recording = True  # Enable recording
    time.sleep(0.05)  # Avoid excessive CPU usage

# Start real-time fNIRS recording
while recording:
    # Check for new markers (to possibly stop recording)
    marker_sample, marker_timestamp = marker_inlet.pull_sample(timeout=0.1)
    if marker_sample and marker_sample[0] != 12:
        print(f"New marker '{marker_sample[0]}' received, stopping recording.")
        break  

    # Receive fNIRS data
    sample, timestamp = fnirs_inlet.pull_sample(timeout=2.0)

    if sample:
        print(f"Timestamp: {timestamp}, Data: {sample}")
        no_data_count = 0  

        num_channels = len(sample) // 4  
        hbo_values = sample[num_channels * 2 : num_channels * 3]  
        hbr_values = sample[num_channels * 3 : num_channels * 4]  

        timestamps.append(timestamp)
        hbo_data.append(np.mean(hbo_values))  
        hbr_data.append(np.mean(hbr_values))

        if len(timestamps) > 100:
            timestamps.pop(0)
            hbo_data.pop(0)
            hbr_data.pop(0)

        # Clear previous plot and update
        clear_output(wait=True)
        ax.clear()
        ax.plot(timestamps, hbo_data, label="HbO", color="red")
        ax.plot(timestamps, hbr_data, label="HbR", color="blue")

        ax.set_xlabel("Time (s)")
        ax.set_ylabel("Concentration (µmol/L)")
        ax.set_title("Real-time fNIRS HbO/HbR Data")
        ax.legend()
        ax.relim()
        ax.autoscale_view()

        display(fig)
        plt.pause(0.01)  

    else:
        no_data_count += 1
        print(f"No data received ({no_data_count}/{max_no_data_attempts}).")

        if no_data_count >= max_no_data_attempts:
            print("No data received multiple times. Exiting script.")
            break  

    time.sleep(0.05)  

plt.close(fig)  
print("Recording stopped. Exiting script.")


In [ ]:
import matplotlib.pyplot as plt
from pylsl import StreamInlet, resolve_byprop
import numpy as np
import time
from IPython.display import display, clear_output

# Look for Aurora fNIRS LSL stream
print("Looking for Aurora fNIRS LSL stream...")

streams = resolve_byprop('name', 'Aurora', timeout=20)
if not streams:
    print("No 'Aurora' LSL stream found. Make sure Aurora is streaming.")
    exit()

# Connect to the LSL stream
inlet = StreamInlet(streams[0])
print(f"Connected to LSL stream: {streams[0].name()}")

# Setup plot
plt.ion()
fig, ax = plt.subplots()
hbo_data, hbr_data, timestamps = [], [], []
no_data_count = 0  
max_no_data_attempts = 5  
data_received = False  # Flag to track if any data has been received

# Wait for first data sample before enforcing exit condition
print("Waiting for first data sample...")
while not data_received:
    sample, timestamp = inlet.pull_sample(timeout=2.0)
    if sample:
        print(f"First data received at {timestamp}, starting real-time plot.")
        data_received = True  # Enable exit condition
    time.sleep(0.05)  # Avoid excessive CPU usage

# Start real-time plotting loop
while True:
    sample, timestamp = inlet.pull_sample(timeout=2.0)

    if sample:
        print(f"Timestamp: {timestamp}, Data: {sample}")
        no_data_count = 0  

        num_channels = len(sample) // 4  
        hbo_values = sample[num_channels * 2 : num_channels * 3]  
        hbr_values = sample[num_channels * 3 : num_channels * 4]  

        timestamps.append(timestamp)
        hbo_data.append(np.mean(hbo_values))  
        hbr_data.append(np.mean(hbr_values))

        if len(timestamps) > 100:
            timestamps.pop(0)
            hbo_data.pop(0)
            hbr_data.pop(0)

        # Clear previous plot and update
        clear_output(wait=True)
        ax.clear()
        ax.plot(timestamps, hbo_data, label="HbO", color="red")
        ax.plot(timestamps, hbr_data, label="HbR", color="blue")

        ax.set_xlabel("Time (s)")
        ax.set_ylabel("Concentration (µmol/L)")
        ax.set_title("Real-time fNIRS HbO/HbR Data")
        ax.legend()
        ax.relim()
        ax.autoscale_view()

        display(fig)
        plt.pause(0.01)  

    else:
        no_data_count += 1
        print(f"No data received ({no_data_count}/{max_no_data_attempts}).")

        if no_data_count >= max_no_data_attempts:
            print("No data received multiple times. Exiting script.")
            break  

    time.sleep(0.05)  

plt.close(fig)  
